# Building Searching System with AgentLite
This tutorial is a simple case on how to design search agent with AgentLite. Designing an agent should follow steps:
- Defining the actions of agents
- Defining agent
- Test agent

## Define your actions
AgentLite provides the BaseAction as a class wrapper for agent to understand how to use the tool. BaseAction has three necessary property
- action_name
- action_desc
- params_doc
Agent will use these three property to understand how to use this action.
And the action execution is implemented in __call__() method. The params_doc is a dictionary to explain the input parameters in __call__() method.

In [1]:
import os
from agentlite.actions.BaseAction import BaseAction
from langchain_community.tools import DuckDuckGoSearchResults, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# define the ducksearch action
class DuckSearch(BaseAction):
    def __init__(self) -> None:
        action_name = "DuckDuckGo_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        self.search = DuckDuckGoSearchResults()
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )

    def __call__(self, query):
        return self.search.run(query)

# define the wikipedia search action
class WikipediaSearch(BaseAction):
    def __init__(self) -> None:
        action_name = "Wikipedia_Search"
        action_desc = "Using this API to search Wiki content."
        params_doc = {"query": "the search string. be simple."}

        self.search = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )

    def __call__(self, query):
        return self.search.run(query)


## Define your search agent
There are two ways for building the agent prompt, w/ example and w/o example
- if w/o example, one just need to provide the name, roles and actions
- if w/ example, one should provide the task-action-observation chain as an example 


### w/o example
the name and role are necessary property for defining an agent

In [2]:
# defining the ducksearch agent as an agent to use DuckSearch API from langchain
from typing import List
from agentlite.actions import BaseAction
from agentlite.agents import ABCAgent, BaseAgent
from agentlite.llm.agent_llms import BaseLLM, get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging.multi_agent_log import AgentLogger

## using the agentrool logger to record the running log
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)

class SearchAgent(BaseAgent):
    def __init__(
        self,
        llm: BaseLLM,
        actions: List[BaseAction] = [DuckSearch(), WikipediaSearch()], # we add both actions into agents
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "search_agent"
        role = """You can answer questions by choosing DuckDuckGo_Search and Wikipedia_Search actions. Finish it if you find the answer."""
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
            logger=agent_logger,
        )

In [3]:
# get the llm for agent
llm_config_dict = {"llm_name": "gpt-3.5-turbo", "temperature": 0.9}
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

In [4]:
# calling the agent with TaskPackage
from agentlite.commons import TaskPackage
search_agent = SearchAgent(llm=llm)
test_task = "what is the found date of microsoft"
test_task_pack = TaskPackage(instruction=test_task)
response = search_agent(test_task_pack)
print("response:", response)


Agent search_agent receives the following TaskPackage:
[
	Task ID: dfdca526-ef96-4df8-8cf4-de2b5edf1cdf
	Instruction: what is the found date of microsoft
]
====search_agent starts execution on TaskPackage dfdca526-ef96-4df8-8cf4-de2b5edf1cdf====


/Users/zhiweiliu/opt/anaconda3/envs/webarena/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Agent search_agent takes 0-step Action:
{
	name: DuckDuckGo_Search
	params: {'query': 'found date of microsoft'}
}
Observation: [snippet: Microsoft is a multinational computer technology corporation. Microsoft was founded on Apr[TLDR]
Agent search_agent takes 1-step Action:
{
	name: Wikipedia_Search
	params: {'query': 'found date of microsoft'}
}
Observation: Page: Microsoft Dynamics 365
Summary: Microsoft Dynamics 365 is a set of enterprise accounting and s[TLDR]
Agent search_agent takes 2-step Action:
{
	name: Finish
	params: {'response': 'Microsoft was founded on April 4, 1975 by Bill Gates and Paul Allen in Albuquerque, New Mexico.'}
}
Observation: "This is the wrong action to call. Please check your available action list.
=========search_agent finish execution. TaskPackage[ID:dfdca526-ef96-4df8-8cf4-de2b5edf1cdf] status:
[
	completion: completed
	answer: Microsoft was founded on April 4, 1975 by Bill Gates and Paul Allen in Albuquerque, New Mexico.
]
response: Microsoft was founde

### w/ example
Providing example for agent in the example will enable more stablity for agent execution. The task-act-obs chain should be provided by users in defining the agent


In [9]:
from agentlite.actions.InnerActions import INNER_ACT_KEY
agent_logger = AgentLogger(OBS_OFFSET=1000, PROMPT_DEBUG_FLAG=False)

class WikiSearchAgent(BaseAgent):
    def __init__(
        self, llm: BaseLLM, actions: List[BaseAction] = [WikipediaSearch()], **kwargs
    ):
        name = "wiki_search_agent"
        role = "You can answer questions by searching wikipedia content. Finish if you find the answer to the task."
        super().__init__(
            name=name, role=role, llm=llm, actions=actions, logger=agent_logger
        )

    def __build_examples__(self):
        """
        constructing the examples for agent working.
        Each example is a successful action-obs chain of an agent.
        those examples should cover all those api calls
        """
        # an example of search agent with wikipedia api call
        # task
        task = "what is the found date of salesforce?"

        # 1. think action and obs
        thought = "I should first use Wikipedia_Search to search salesforce"
        act_1 = AgentAct(name=ThinkAct.action_name, params={INNER_ACT_KEY: thought})
        obs_1 = ""

        # 2. wikipedia_search action and obs
        act_params = {"query": "salesforce"}
        act_2 = AgentAct(name=WikipediaSearch().action_name, params=act_params)
        obs_2 = """Page: Salesforce
        Summary: Salesforce, Inc. is an American cloud-based software company headquartered in San Francisco, California. It provides customer relationship management (CRM) software and applications focused on sales, customer service, marketing automation, e-commerce, analytics, and application development.
        Founded by former Oracle executive Marc Benioff in February 1999, Salesforce grew quickly, making its IPO in 2004. As of September 2022, Salesforce is the 61st largest company in the world by market cap with a value of nearly US$153 billion. Salesforce's rapid growth made it the first cloud computing company to reach US$1 billion in annual revenue, which it achieved in fiscal year 2009. It became the world's largest enterprise software firm in 2022. Salesforce ranked 136th on the most recent edition of the Fortune 500, making US$26.5 billion in 2022. Since 2020, Salesforce has also been a component of the Dow Jones Industrial Average."""

        # 3. think action and obs
        thought = "I find salesforce is Founded by former Oracle executive Marc Benioff in February 1999"
        act_3 = AgentAct(name=ThinkAct.action_name, params={INNER_ACT_KEY: thought})
        obs_3 = ""

        # 4. finish action
        answer = "February 1999"
        act_4 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: answer})
        obs_4 = "Task Completed."
        
        # 5. adding into prompt_gen of the agent

        task_pack = TaskPackage(instruction=task)
        act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3), (act_4, obs_4)]
        self.add_example(task=task_pack, action_chain=act_obs)

In [10]:
# calling the agent with TaskPackage
from agentlite.commons import TaskPackage
search_agent = WikiSearchAgent(llm=llm)
test_task = "what is the found date of microsoft"
test_task_pack = TaskPackage(instruction=test_task)
response = search_agent(test_task_pack)
print("response:", response)

Agent wiki_search_agent receives the following TaskPackage:
[
	Task ID: dfdca526-ef96-4df8-8cf4-de2b5edf1cdf
	Instruction: what is the found date of microsoft
]
====wiki_search_agent starts execution on TaskPackage dfdca526-ef96-4df8-8cf4-de2b5edf1cdf====
Agent wiki_search_agent takes 0-step Action:
{
	name: Wikipedia_Search
	params: {'query': 'microsoft history'}
}
Observation: Page: History of Microsoft
Summary: Microsoft is a multinational computer technology corporation. Microsoft was founded on April 4, 1975, by Bill Gates and Paul Allen in Albuquerque, New Mexico. Its current best-selling products are the Microsoft Windows operating system; Microsoft Office, a suite of productivity software; Xbox, a line of entertainment of games, music, and video; Bing, a line of search engines; and Microsoft Azure, a cloud services platform.In 1980, Microsoft formed a partnership with IBM to bundle Microsoft's operating system with IBM computers; with that deal, IBM paid Microsoft a royalty for